# Pytorch VGG16（Cat and Dog）初心者の視点で実装したバージョン


【お題】
- 検証用データセット「Cat and Dog」を用いて画像分類のための機械学習モデルを構築する。
 - 「Cat and Dog」データセット
   - 画像数：16010(学習：12808、検証：3202）
   - 画像サイズ：64×64
   - クラス数：2（cats、dogs）
  - データセットの入手先：
   - https://www.kaggle.com/tongpython/cat-and-dog
    - cat-and-dog.zip

【提出物】
- モデル学習時のepochごとのAccurncy、loss値の推移が分かるもの。
- モデル構築後、検証用画像に対してモデルを適用した際の、Accurncyが分かるもの。

## ■目次

- モジュールの読み込み等
- Google Colaboratory を使うための準備
- 入力データ(学習、評価) 整備
- モデル定義
- 学習
- 評価

## ■モジュールの読み込み等

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary # summary(your_model, input_size=(channels, H, W))

import sys
import os
import shutil
import io
import numpy as np
import pandas as pd

import random
from datetime import datetime
import json
import pickle
import math

import requests # URLリクエストを簡単に行う。pip install requests が必要
from urllib import request # 指定URLのファイルを保存する
import zipfile
from PIL import Image
import glob

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import confusion_matrix

print(torch.__version__) 

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

In [0]:
# 割当てられたGPUの確認
!nvidia-smi

## ■Google Colaboratory を使うための準備

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/'

if not os.path.exists('tmp'): # 12時間ルール対策用フォルダ
    os.makedirs('tmp')

# 【使用例】
#sys.stdout = open('/content/drive/My Drive/tmp/result.txt', 'a')
# …とても時間のかかる処理（学習処理など）…
#sys.stdout.close()
#sys.stdout = sys.__stdout__

## ■入力データ(学習、評価) 整備

1. torchvision.datasetsで入手できるもの（MNIST, CIFAR10など）はこちらを使う。
2. torchvision.datasetsで入手できないものは、Kaggle等から入手したデータセット（ZIP）をGoogle Driveに予めアップロードしておく。
 - データセット仕様（フォルダ構成など）は、学習(train)と評価(val)データの振り分け処理のために、事前に把握しておくこと。
 - Sign Language MNISTなど、画像ファイルでなくCSVで提供されるデータセットは自分で画像ファイルに変換しZIP化したものをアップロードする。



### データセットをColabo VMに展開

In [0]:
gdrive_dir = "/content/drive/My Drive/Colab Notebooks/dataset/"
download_dir = "/root/download/"
data_dir = "/root/data/"
zip_file_name = "cat-and-dog.zip"

!rm -rf $data_dir

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [0]:
# Colab VMのストレージにDLする
%%time
s = gdrive_dir + zip_file_name
shutil.copy(s, download_dir)

In [0]:
# Colab VMのストレージに解凍する
%%time
f = os.path.join(download_dir, zip_file_name)
with zipfile.ZipFile(f) as zip:
  zip.extractall(download_dir)

In [0]:
os.chdir(download_dir)
%pwd

In [0]:
%ls

### 学習データ(train)と評価データ(validation)の振分け

- 本ファイルが扱うデータセットの学習データと評価データは、解凍直後の段階で既に、におよそ 75％：25％ の割合でフォルダ別に仕切られている。
- この割合で学習を評価を行うことにする。

#### 学習データの振分け

In [0]:
# ラベル名を確認する
src_dir = '/root/download/training_set/training_set/'
dst_dir = '/root/data/cat_and_dog/train/'

!rm -rf $dst_dir
os.makedirs(dst_dir)

os.chdir(src_dir)
label_dirs = os.listdir(path='.')
print(label_dirs)

In [0]:
# 所定フォルダに振り分ける
for dir_name in label_dirs:
  numof_data = len( os.listdir(src_dir + dir_name)) # ラベルフォルダ内のファイル数
  files = glob.glob(src_dir + dir_name + "/*.jpg")  # ラベルフォルダ内のファイルリスト

  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    d = dst_dir + dir_name # 振り分け先フォルダ
    if not os.path.exists(d):
      os.makedirs(d)
    shutil.move(file, d)

In [0]:
# 意図通りに振り分けられたか確認する（今回は目視で確認）
print("Train files in " + dst_dir)
for dir_name in label_dirs:
  numof_data = len( os.listdir(dst_dir + dir_name)) # ラベルフォルダ内のファイル数
  print(" " + dir_name + ": " + str(numof_data))

#### 評価データの振分け

In [0]:
# ラベル名を確認する
src_dir = '/root/download/test_set/test_set/'
dst_dir = '/root/data/cat_and_dog/test/'

!rm -rf $dst_dir
os.makedirs(dst_dir)

os.chdir(src_dir)
label_dirs = os.listdir(path='.')
print(label_dirs)

In [0]:
# 所定フォルダに振り分ける
for dir_name in label_dirs:
  numof_data = len( os.listdir(src_dir + dir_name)) # ラベルフォルダ内のファイル数
  files = glob.glob(src_dir + dir_name + "/*.jpg")  # ラベルフォルダ内のファイルリスト

  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    d = dst_dir + dir_name # 振り分け先フォルダ
    if not os.path.exists(d):
      os.makedirs(d)
    shutil.move(file, d)

In [0]:
# 意図通りに振り分けられたか確認する（今回は目視で確認）
print("Validation files in " + dst_dir)
for dir_name in label_dirs:
  numof_data = len( os.listdir(dst_dir + dir_name)) # ラベルフォルダ内のファイル数
  print(" " + dir_name + ": " + str(numof_data))

### データセット作成

In [0]:
dst_dir = '/root/data/cat_and_dog/'

In [0]:
# VGGモデル入力を行うための画像のフォーマット変換
my_size = 224
my_mean = (0.485, 0.456, 0.406) # ILSVRC2012に基づくデータセットを使う場合の設定値
my_std = (0.229, 0.224, 0.225) # ILSVRC2012に基づくデータセットを使う場合の設定値

In [0]:
tf_train = transforms.Compose([
     transforms.Resize((my_size, my_size)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomRotation(90),
     transforms.ToTensor(),
     transforms.Normalize(my_mean, my_std)])

train_dataset = datasets.ImageFolder(root = dst_dir + 'train', transform=tf_train) 
train_dataset

In [0]:
tf_val = transforms.Compose([
     transforms.Resize((my_size, my_size)),
     transforms.ToTensor(),
     transforms.Normalize(my_mean, my_std)])

val_dataset = datasets.ImageFolder(root = dst_dir + 'test', transform=tf_val)
val_dataset

In [0]:
print(train_dataset.classes)
print(val_dataset.classes)

### データローダ作成

In [0]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                        batch_size = 64, shuffle = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                        batch_size = 64, shuffle = True)

## ■モデル定義

In [0]:
# モデルDL
net = models.vgg16(pretrained = True)

# アーキ変更
# 【変更前】(fc): Linear(in_features=512, out_features=1000, bias=True)
# 【変更後】(fc): Linear(in_features=512, out_features=3, bias=True)
net.fc = nn.Linear(512, 3) # fc層を置き換える

# 意図通りに変更できたかどうかを目視で確認
for name, param in net.named_parameters():
  print(f'name={name}, param={param.shape}')

In [0]:
# 指定層の重みを学習中に変更できるようにする。

# 転移学習（出力層）
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# ファインチューニング（出力層以外）
IS_ENABLE_FINETUNING = 0 # ファインチューニングしない場合は「0」
update_param_names_fine = ["features","classifier.0.weight","classifier.0.bias","classifier.3.weight","classifier.3.bias"]

if bool(IS_ENABLE_FINETUNING):
  update_param_names = update_param_names_fine + update_param_names

params_to_update = []
for name, param in net.named_parameters():
  if  "features" in name and "features" in update_param_names:
    param.requires_grad = True
    params_to_update.append(param)
    print(f'{name} is tunable.')
  elif name in update_param_names:
    param.requires_grad = True
    params_to_update.append(param)
    print(f'{name} is tunable.')
  else:
    param.requires_grad = False # 指定外層の重みは学習で変更されないようにする

In [0]:
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)
model = net.to(device)
summary(model, input_size=(3,my_size, my_size))

## ■学習

In [0]:
%%time
sys.stdout = open('/content/drive/My Drive/tmp/result.txt', 'w')
model.train() # 再学習モードに設定する
criterion = nn.CrossEntropyLoss()
epochs = 50
running_loss_history = [] # 学習時の損失値の履歴
running_corrects_history = [] # 学習時の正解率の履歴
val_running_loss_history = [] # 評価時の損失値の履歴
val_running_corrects_history = [] # 評価時の正解率の履歴

for e in range(epochs):
  running_loss = 0.0
  running_corrects = 0.0
  val_running_loss = 0.0
  val_running_corrects = 0.0
  
  for inputs, labels in train_dataloader:
    model.train()
    outputs = model(inputs.to(device))
    loss = criterion(outputs, labels.to(device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, preds = torch.max(outputs, 1)
    running_loss += loss.item()
    running_corrects += torch.sum(preds == (labels.data).to(device))
  
  else:
    with torch.no_grad():
      model.eval()
      true_label_per_epoch, pred_label_per_epoch = [],[]

      for val_inputs, val_labels in val_dataloader:
        val_outputs = model(val_inputs.to(device))
        val_loss = criterion(val_outputs, val_labels.to(device))

        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds == (val_labels.data).to(device))

        true_label_per_epoch += (val_labels.numpy()).tolist()
        pred_label_per_epoch += (val_preds.to("cpu").numpy()).tolist()

#   学習過程を記録
    epoch_loss = running_loss/len(train_dataloader.dataset)
    epoch_acc = running_corrects.float()/ len(train_dataloader.dataset)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    
    val_epoch_loss = val_running_loss/len(val_dataloader.dataset)
    val_epoch_acc = val_running_corrects.float()/len(val_dataloader.dataset)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)
    
    print('epoch *', (e+1))
    print(f'training loss: {epoch_loss:.4f} (={running_loss:.2f}/{len(train_dataloader.dataset)}), \
    training acc {epoch_acc.item():.4f} (={running_corrects:.2f}/{len(train_dataloader.dataset)})')
    print(f'validation loss: {val_epoch_loss:.4f} (={val_running_loss:.2f}/{len(val_dataloader.dataset)}), \
    validation acc {val_epoch_acc.item():.4f} (={val_running_corrects:.2f}/{len(val_dataloader.dataset)})') 


In [0]:
import sys
sys.stdout.close()
sys.stdout = sys.__stdout__

## ■評価

In [0]:
# 学習精度曲線（青）、テスト精度曲線（橙）
plt.plot(running_corrects_history, label='training accuracy')
plt.plot(val_running_corrects_history, label='validation accuracy')
plt.legend()

In [0]:
# ロス（誤差）関数曲線
plt.plot(running_loss_history, label='training loss')
plt.plot(val_running_loss_history, label='validation loss')
plt.legend()